In [ ]:
import os
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import joblib

# Load model
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0)

def load_cropface_embeddings(folder):
    embeddings = []
    labels = []

    for filename in os.listdir(folder):
        if not filename.lower().endswith(('.jpg', '.png')):
            continue

        label = filename.split('_')[0]  # Lấy tên trước dấu _
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Không đọc được ảnh: {img_path}")
            continue

        img = cv2.resize(img, (112, 112))  # đảm bảo đúng input size
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Trích embedding từ ảnh đã crop
        embedding = app.models['recognition'].get_feat(img).flatten()
        embeddings.append(embedding)
        labels.append(label)

    return np.array(embeddings), np.array(labels)

# Bước 1: Load ảnh và trích embedding
X, y = load_cropface_embeddings("facecrop")
print(f"✅ Số ảnh: {len(X)}, nhãn: {set(y)}")

if len(X) < 2 or len(set(y)) < 2:
    raise ValueError("⚠️ Không đủ dữ liệu để train SVM. Cần ít nhất 2 ảnh từ 2 nhãn khác nhau.")

# Bước 2: Encode nhãn
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Bước 3: Train SVM
clf = SVC(kernel='linear', probability=True)
clf.fit(X, y_encoded)

# Bước 4: Lưu model
joblib.dump(clf, "svm_face_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("✅ Đã train & lưu model.")

# OMG IT WORKEDDDDDDDD AJNWDJNAWDJKNAWJDNKAWKJDAWNJDAWKJNDAWJKDJAKWDANWKJDJKNAWKDNJAWNJK


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
✅ Số ảnh: 49

/opt/conda/lib/python3.10/site-packages/sklearn/utils/multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  y_type = type_of_target(y, input_name="y")


✅ Đã train & lưu model.


In [2]:
print("hi")

hi
